In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/job-resume/resume_job_description_fit_trains_matching.csv


In [2]:
df=pd.read_csv('/kaggle/input/job-resume/resume_job_description_fit_trains_matching.csv')

In [3]:
df.head()

,resume_text,job_description_text,label
0,SummaryHighly motivated Sales Associate with e...,Net2Source Inc. is an award-winning total work...,No Fit
1,Professional SummaryCurrently working with Cat...,At Salas OBrien we tell our clients that were ...,No Fit
2,SummaryI started my construction career in Jun...,Schweitzer Engineering Laboratories (SEL) Infr...,No Fit
3,SummaryCertified Electrical Foremanwith thirte...,"Mizick Miller & Company, Inc. is looking for a...",No Fit
4,SummaryWith extensive experience in business/r...,Life at Capgemini\nCapgemini supports all aspe...,No Fit


In [4]:
df.shape

(6241, 3)

In [5]:
df.isna().sum()

resume_text             0
job_description_text    0
label                   0
dtype: int64

In [6]:
df.duplicated().sum()

1

In [7]:
# remove duplicates
df= df.drop_duplicates()

In [8]:
df.duplicated().sum()

0

In [9]:
df['label'].value_counts()

label
No Fit           3142
Potential Fit    1556
Good Fit         1542
Name: count, dtype: int64

In [10]:
# number of category
df['label'].value_counts()
print(df['label'].value_counts())

# check the maximum value counts
print('maximum',df['label'].value_counts().max())
max_size=df['label'].value_counts().max()
# upsampling for balanced the classes
balanced_df=df.groupby('label').apply(lambda x: x.sample(max_size,replace=True)).reset_index(drop=True)

# check the balance class
print('balanced class--',balanced_df['label'].value_counts())

# shuffle the data to avoid any order
df=balanced_df.sample(frac=1).reset_index(drop=True)  # frac1 100% rows are shuffled

label
No Fit           3142
Potential Fit    1556
Good Fit         1542
Name: count, dtype: int64
maximum 3142
balanced class-- label
Good Fit         3142
No Fit           3142
Potential Fit    3142
Name: count, dtype: int64


In [11]:
import re
import string

def clean_text(txt):
    # Remove URLs
    pattern = re.compile(r'https?://\S+|www\.\S+')
    txt = pattern.sub(r'', txt)
    
    # Remove RT and cc
    txt = re.sub(r'\b(RT|cc)\b', ' ', txt)
    
    # Remove Hashtags
    txt = re.sub(r'#\S+\s', '', txt)
    
    # Remove Mentions
    txt = re.sub(r'@\S+', '', txt)
    
    # Remove Punctuation
    exclude = string.punctuation
    txt = txt.translate(str.maketrans('', '', exclude))
    
    # Remove Non-ASCII Characters
    txt = re.sub(r'[^\x00-\x7f]', ' ', txt)
    
    # Normalize Whitespace
    txt = re.sub('\s+', ' ', txt).strip()
    
    return txt

In [12]:
df[['resume_text', 'job_description_text']] = df[['resume_text', 'job_description_text']].apply(lambda x: x.apply(clean_text))

In [13]:
res=df[['resume_text','job_description_text']]

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Combine resume and job description text into one list
questions = list(df['resume_text']) + list(df['job_description_text'])

# Initialize TfidfVectorizer
tfidf = TfidfVectorizer(max_features=10000)

# Fit and transform the combined text data
questions_tfidf = tfidf.fit_transform(questions).toarray()

# Split the transformed array into two parts: one for resume_text and one for job_description_text
q1_arr, q2_arr = np.vsplit(questions_tfidf, 2)

# Verify shapes of the resulting arrays
print("Shape of q1_arr (resume_text):", q1_arr.shape)
print("Shape of q2_arr (job_description_text):", q2_arr.shape)


Shape of q1_arr (resume_text): (9426, 10000)
Shape of q2_arr (job_description_text): (9426, 10000)


In [15]:
temp_df1=pd.DataFrame(q1_arr,index=df.index)
temp_df2=pd.DataFrame(q2_arr,index=df.index)
temp_df=pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(9426, 20000)

In [16]:
temp_df['label']=df['label']

In [17]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
temp_df['label']=lb.fit_transform(temp_df['label'])

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [19]:
x_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.06816864,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [20]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Ensure that X_train and X_test are dense if they are sparse
x_train = x_train.toarray() if hasattr(x_train, 'toarray') else x_train

# 1. Train RandomForestClassifier
x_test = x_test.toarray() if hasattr(x_test, 'toarray') else x_test

rf_model = OneVsRestClassifier(RandomForestClassifier())
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_test)
print("\nRandomForestClassifier Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_rf)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_rf)}")



RandomForestClassifier Results:
Accuracy: 0.9093
Confusion Matrix:
[[626   7   7]
 [ 60 505  55]
 [ 17  25 584]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.98      0.93       640
           1       0.94      0.81      0.87       620
           2       0.90      0.93      0.92       626

    accuracy                           0.91      1886
   macro avg       0.91      0.91      0.91      1886
weighted avg       0.91      0.91      0.91      1886



In [21]:
xgb_model = OneVsRestClassifier(xgb.XGBClassifier())
xgb_model.fit(x_train, y_train)
y_pred_xgb = xgb_model.predict(x_test)
print("\nXGBClassifier Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred_xgb)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred_xgb)}")


XGBClassifier Results:
Accuracy: 0.9136
Confusion Matrix:
[[619  15   6]
 [ 51 519  50]
 [ 19  22 585]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       640
           1       0.93      0.84      0.88       620
           2       0.91      0.93      0.92       626

    accuracy                           0.91      1886
   macro avg       0.91      0.91      0.91      1886
weighted avg       0.91      0.91      0.91      1886



In [22]:
def pred(input_resume, input_job_description):
    # Preprocess both the resume and job description
    cleaned_resume = clean_text(input_resume)
    cleaned_job_desc = clean_text(input_job_description)
    
    # Vectorize the cleaned texts using the TF-IDF vectorizer
    # We assume `q1_arr` corresponds to the resume features and `q2_arr` corresponds to job description features
    vectorized_resume = tfidf.transform([cleaned_resume]).toarray()
    vectorized_job_desc = tfidf.transform([cleaned_job_desc]).toarray()
    
    # Concatenate the vectors from both the resume and job description
    combined_vector = np.concatenate((vectorized_resume, vectorized_job_desc), axis=1)
    
    # Make prediction using the model (xgb_model)
    predicted_label = xgb_model.predict(combined_vector)
    # print("pr"+predicted_label)
    
    # Convert the predicted label to its corresponding category name
    predicted_label_name = lb.inverse_transform(predicted_label)
    
    return predicted_label_name[0]



In [23]:
resume = """

Here's an example of a resume that could align well with the job description you provided for the Data Scientist Intern role. This resume emphasizes the relevant skills and experiences while keeping the language focused on the job's key requirements:

SAHIL YADAV
Chhpara Salimpur, District-Mohindergarh, Haryana
Email: sahilchhapra01@gmail.com
LinkedIn: linkedin.com/in/sahilyadav
GitHub: github.com/sahilyadav

Objective
A passionate and driven Computer Science Engineering student with a strong foundation in Data Science, Machine Learning, and Python programming. Seeking to leverage my skills in data analysis, machine learning, and statistical modeling in a Data Scientist Intern position at OptimSpace to gain hands-on experience and contribute to data-driven solutions.

Education
B.Tech in Computer Science Engineering (CSE)
JECRCE University
Expected Graduation: 2026
CGPA: 9.35 (First Year)

Key Skills
Programming Languages: Python, Java, SQL
Data Analysis & Visualization: Pandas, NumPy, Matplotlib, Seaborn
Machine Learning: Scikit-learn, XGBoost, Keras, TensorFlow
Statistical Modeling: Regression, Classification, Clustering
Data Preprocessing: Data cleaning, feature extraction, handling missing data
Database Management: SQL, MySQL
Tools & Platforms: Jupyter, Google Cloud, Databricks, Streamlit
Soft Skills: Critical thinking, problem-solving, teamwork, communication
Projects
Stock Price Prediction (LSTM Model)
Developed a time-series prediction model using LSTM to forecast stock prices based on historical data.
Implemented data preprocessing techniques for handling missing values, scaling, and feature extraction.
Credit Risk Modeling
Built a robust credit risk model using XGBoost to predict loan defaults, providing insights for better financial decision-making.
Applied data preprocessing, feature engineering, and model evaluation techniques for improved performance.
Resume Screening Platform (AI-based)
Designed an NLP-based resume screening system that automates candidate selection by analyzing resumes and matching them with job descriptions.
Implemented text classification, keyword extraction, and semantic analysis using spaCy and NLTK.
Chest X-Ray Classification (Deep Learning)
Created a deep learning model for classifying chest X-rays into categories like viral pneumonia, bacterial pneumonia, and tuberculosis using CNNs.
Deployed the model using TensorFlow and Python.
Personality Analysis Model
Developed a personality analysis model to assess and understand traits across different departments to optimize team dynamics and recruitment processes.
Deployed the model on Streamlit for easy access by the HR team.
Internship Experience
MS FINCAP (NBFC)
October 2023 - Ongoing

Developed AI-driven tools for detecting fake videos and voice cloning, improving security and preventing fraud.
Collaborated with data scientists to analyze financial data and build predictive models for loan approval.
Certifications
Google Cloud Certified: Google Cloud Foundation
Databricks Fundamentals Certificate
Participated in Hackathons: Data Science & Innovation (NPTEL)
Extracurricular Activities
Hackathon Winner: Developed a Machine Learning Model for stock price prediction.
Volunteer: Assisted in organizing tech workshops and seminars for beginners in data science and programming.
Languages
English: Fluent
Hindi: Fluent
Hobbies
Reading: Exploring new technologies and advancements in AI/ML.
Coding: Passionate about learning new programming languages and building projects.
Gaming: Engaging in strategy games to enhance problem-solving skills

"""


In [24]:
job_description = """

Skills:
Data Analysis, Statistical Modeling, Machine Learning, Python Programming, Data Visualization, SQL, Critical Thinking, Problem Solving,

Job Overview

OptimSpace is seeking a highly motivated Data Scientist Intern to join our team. This is a remote internship opportunity available to freshers with a passion for data science. Locations include Noida, Hyderabad, Chennai, Gurgaon, Pune, Bangalore, and Mumbai. The position is a perfect fit for individuals ready to craft their skills in data analysis and enhance their career pathway in the technology industry.

Qualifications And Skills

Proficiency in data analysis to interpret and derive meaningful insights from complex data sets (Mandatory skill).
Strong Python programming skills for data manipulation, analysis, and model building (Mandatory skill).
Knowledge of SQL for efficient querying and managing of databases (Mandatory skill).
Experience in statistical modeling to help understand trends and make predictions from data.
Understanding of machine learning concepts and algorithms to drive business solutions.
Ability to create data visualizations to effectively communicate findings and insights.
Critical thinking capability to analyze facts for informed decision-making processes.
Excellent problem solving skills to tackle challenges through analytical approaches.

Roles And Responsibilities

Collaborate with the data team to perform data cleaning, preprocessing, and analysis tasks.
Assist in building predictive models and algorithms to support business objectives.
Develop and implement data visualization tools to provide insights to stakeholders.
Support the development of data-driven strategies by formulating novel data analysis techniques.
Contribute to improving data collection procedures to include information relevant for building analytic systems.
Work under the guidance of senior data scientists to gain practical knowledge and experience.
Document processes and findings to contribute to a cohesive data knowledge base.
Participate in training sessions and feedback loops to enhance data competency skills.
Desired Skills and Experience
Data Analysis, Statistical Modeling, Machine Learning, Python Programming, Data Visualization, SQL, Critical Thinking, Problem Solving


"""


In [27]:
pred(resume,job_description)

'No Fit'

In [26]:
print(lb.classes_)

['Good Fit' 'No Fit' 'Potential Fit']


In [28]:
import pickle
pickle.dump(tfidf,open('tfidf1.pkl','wb'))
pickle.dump(xgb_model, open('xgb.pkl', 'wb'))
pickle.dump(lb, open("encoder1.pkl",'wb'))